In [1]:
import numpy as np
import keras
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, concatenate, Flatten, AveragePooling2D
from keras.layers import*
from keras.models import Model
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.optimizers import*
from keras.regularizers import l2
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function
import time

Using Theano backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator

In [13]:
max_epoch = 40
def poly_decay(epoch):
    base_lr = 0.01
    lr = base_lr*((1-(epoch/max_epoch))**0.5)
    return lr
lrate = LearningRateScheduler(poly_decay)
callbacks_list = [lrate]

In [6]:
train_datagen =  ImageDataGenerator(rotation_range=5, width_shift_range=0.08, shear_range=0.1,
                               height_shift_range=0.1, zoom_range=0.1)
val_datagen =  ImageDataGenerator(rotation_range=5, width_shift_range=0.08, shear_range=0.1,
                               height_shift_range=0.1, zoom_range=0.1)

In [7]:
path_train = "fer2013/train/"
path_val = "fer2013/val/"

In [8]:
train_generator = train_datagen.flow_from_directory(
        path_train,
        target_size=(40, 40),
        batch_size=64,
        class_mode='categorical',
        color_mode = 'grayscale')

validation_generator = val_datagen.flow_from_directory(
        path_val,
        target_size=(40, 40),
        batch_size=64,
        class_mode='categorical',
        color_mode = 'grayscale')

Found 287100 images belonging to 7 classes.
Found 35890 images belonging to 7 classes.


In [9]:
img_input = Input(shape=(40,40,1), name='img_input')
x = Conv2D(64,7, strides=(2,2), activation = 'relu',padding='same')(img_input)
x = MaxPooling2D(pool_size=(3, 3), strides=(2,2),padding='same')(x)   
x = Conv2D(192,3,strides=(2,2),padding='same')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2,2),padding='same')(x)
x = BatchNormalization()(x)
#x = Dropout(0.2)(x)

#first inception layer 3a
x1 = Conv2D(64,1, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x) #1*1
x2 = Conv2D(96,3, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x) #3*3 reduce
x2 = Conv2D(128,3, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x2) #3*3 
x3 = Conv2D(16,5, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x) #5*5 reduce
x3 = Conv2D(32,5, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x3) #5*5 
x4 = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same')(x) #pool proj
x4 = Conv2D(32,1, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x4) #1*1
x = concatenate([x1,x2,x3,x4], axis = 3)
x = BatchNormalization()(x)
#x = Dropout(0.2)(x)
#second inception layer 3b
x1 = Conv2D(128,1, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x) #1*1
x2 = Conv2D(128,3,padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x) #3*3 reduce
x2 = Conv2D(192,3, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x2) #3*3 
x3 = Conv2D(32,5, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x) #5*5 reduce
x3 = Conv2D(96,5, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x3) #5*5 
x4 = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same')(x) #pool proj
x4 = Conv2D(64,1, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x4)
x = concatenate([x1,x2,x3,x4], axis = 3)
x = MaxPooling2D(pool_size=(3, 3), strides=(2,2),padding='same')(x)
x = BatchNormalization()(x)
#x = Dropout(0.2)(x)
#third inception layer 4a
x1 = Conv2D(192,1, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x) #1*1
x2 = Conv2D(96,3,padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x) #3*3 reduce
x2 = Conv2D(208,3, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x2) #3*3 
x3 = Conv2D(16,5, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x) #5*5 reduce
x3 = Conv2D(48,5, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x3) #5*5 
x4 = MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same')(x) #pool proj
x4 = Conv2D(64,1, padding='same', activation = 'relu', W_regularizer=l2(0.0002))(x4) #1*1
x = concatenate([x1,x2,x3,x4], axis = 3)
x = BatchNormalization()(x)
x = AveragePooling2D()(x)
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(4096, activation='relu', W_regularizer=l2(0.0002))(x)
x = Dense(1024, activation='relu', W_regularizer=l2(0.0002))(x)
x = Dense(7, activation='softmax', W_regularizer=l2(0.0002))(x)
final_model = Model(inputs = [img_input], outputs = [x])
final_model.summary()

/home/amitoj/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 1, padding="same", activation="relu", kernel_regularizer=<keras.reg...)`
  # Remove the CWD from sys.path while we load stuff.
/home/amitoj/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, 3, padding="same", activation="relu", kernel_regularizer=<keras.reg...)`
  # This is added back by InteractiveShellApp.init_path()
/home/amitoj/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, padding="same", activation="relu", kernel_regularizer=<keras.reg...)`
  if sys.path[0] == '':
/home/amitoj/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, 5, padding="same", activation="relu", kernel_regular

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img_input (InputLayer)          (None, 40, 40, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 20, 20, 64)   3200        img_input[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 10, 10, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 5, 5, 192)    110784      max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
max_poolin

Total params: 8,584,999
Trainable params: 8,582,119
Non-trainable params: 2,880
__________________________________________________________________________________________________


In [10]:
sgd = SGD(lr=0.01, nesterov=False)
final_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [16]:
final_model.load_weights("weights/aug-batchnorm-0.2dropout-01-0.3436.hdf5")

In [17]:
filepath= "weights/" +"aug-batchnorm-0.2dropout" + "-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max')
callbacks_list.append(checkpoint)
model_history = final_model.fit_generator(train_generator, train_generator.samples/train_generator.batch_size,
        validation_data = validation_generator, epochs = max_epoch,
        validation_steps = validation_generator.samples/validation_generator.batch_size,
                                         callbacks = callbacks_list)

Epoch 1/40
4486/4485 [==============================] - 1967s 438ms/step - loss: 1.5674 - acc: 0.3507 - val_loss: 1.6234 - val_acc: 0.3364

Epoch 00001: saving model to weights/aug-batchnorm-0.2dropout-01-0.3364.hdf5

Epoch 00001: saving model to weights/aug-batchnorm-0.2dropout-01-0.3364.hdf5

Epoch 00001: saving model to weights/aug-batchnorm-0.2dropout-01-0.3364.hdf5
Epoch 2/40
4486/4485 [==============================] - 1955s 436ms/step - loss: 1.4690 - acc: 0.4147 - val_loss: 1.3993 - val_acc: 0.4506

Epoch 00002: saving model to weights/aug-batchnorm-0.2dropout-02-0.4506.hdf5

Epoch 00002: saving model to weights/aug-batchnorm-0.2dropout-02-0.4506.hdf5

Epoch 00002: saving model to weights/aug-batchnorm-0.2dropout-02-0.4506.hdf5
Epoch 3/40
4486/4485 [==============================] - 1925s 429ms/step - loss: 1.3700 - acc: 0.4618 - val_loss: 1.5037 - val_acc: 0.4155

Epoch 00003: saving model to weights/aug-batchnorm-0.2dropout-03-0.4155.hdf5

Epoch 00003: saving model to weights

4486/4485 [==============================] - 2080s 464ms/step - loss: 0.8120 - acc: 0.7239 - val_loss: 1.3247 - val_acc: 0.5589

Epoch 00023: saving model to weights/aug-batchnorm-0.2dropout-23-0.5589.hdf5

Epoch 00023: saving model to weights/aug-batchnorm-0.2dropout-23-0.5589.hdf5

Epoch 00023: saving model to weights/aug-batchnorm-0.2dropout-23-0.5589.hdf5
Epoch 24/40
4486/4485 [==============================] - 2083s 464ms/step - loss: 0.7956 - acc: 0.7319 - val_loss: 1.2991 - val_acc: 0.5775

Epoch 00024: saving model to weights/aug-batchnorm-0.2dropout-24-0.5775.hdf5

Epoch 00024: saving model to weights/aug-batchnorm-0.2dropout-24-0.5775.hdf5

Epoch 00024: saving model to weights/aug-batchnorm-0.2dropout-24-0.5775.hdf5
Epoch 25/40
4486/4485 [==============================] - 2084s 465ms/step - loss: 0.7747 - acc: 0.7406 - val_loss: 1.3508 - val_acc: 0.5596

Epoch 00025: saving model to weights/aug-batchnorm-0.2dropout-25-0.5596.hdf5

Epoch 00025: saving model to weights/aug-batc

KeyboardInterrupt: 

Epoch 1/40
 145/4485 [..............................] - ETA: 31:30 - loss: 1.9118 - acc: 0.2407

KeyboardInterrupt: 

In [ ]:
plt.figure()
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.savefig("bn0.2.jpg")
plt.show()